In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
import sys
from pathlib import Path

import ipywidgets as w
import ipyvuetify as v
import pdblp
import numpy as np
import pandas as pd
from IPython.display import display
from ipydatagrid import DataGrid
from sklearn.linear_model import LinearRegression
from plotly import graph_objects as go, express as px

root_path = Path('/dev/projects/credit/backend/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)
    
import utils
from api.data.processing import get_current_data
from api.gui.base import Tab

In [ ]:
bonds_data = get_current_data()

In [ ]:
currency = 'USD'
capital_stack = 'at1'

data_ = bonds_data.query(f"currency=='{currency}' and capital_stack=='{capital_stack}'")

In [ ]:
columns = [
    'name', 
    'issuer_equity_ticker', 
    'ytm_flat', 
    'years_to_maturity'
]
plot_df = data_[columns]

In [ ]:
fig = px.scatter(
    plot_df,
    x='years_to_maturity',
    y='ytm_flat', 
    color='issuer_equity_ticker',
    hover_name='name',
    height=1000, 
    template='plotly_white',
    text='name'
)

fig.update_traces(
    textposition='bottom center',
    textfont_size=8
)

In [ ]:
data_.query('ticker == "HSBC"')

In [ ]:
fig.update_traces(
    textposition='bottom center',
    textfont_size=8
)

In [ ]:
group_sel = v.Autocomplete(
    v_model='issuer_name',
    items=[
        'issuer_name',
        'bics_sector',
        'bics_industry',
        'country',
        'currency',
        'rating_bucket',
        'area',
        'macroarea'
    ],
    label='group',
    dense=True,
    outlined=True
)

cap_stack_sel = v.Autocomplete(
    v_model='at1',
    items=['at1', 'lt2', 'sp', 'snp'],
    label='capital stack',
    dense=True,
    outlined=True
)


curr_sel = v.Autocomplete(
    v_model='EUR',
    items=['EUR', 'USD', 'GBP'],
    label='currency',
    dense=True,
    outlined=True
)


x_sel = v.Autocomplete(
    v_model='years_to_maturity',
    items=[
        'years_to_maturity', 
        'mod_dur', 
    ],
    label='x',
    dense=True,
    outlined=True
)

y_sel = v.Autocomplete(
    v_model='ytm_flat',
    items=[
        'yield_to_call', 
        'ytm_flat', 
        'ytm_fwd', 
        'yield_to_worst',
        'xccy_usd', 
        'i_spread'
        'z_spread', 
        'aws_spread', 
        'z_spread_basis', 
        'reset_spread',
        'yield_to_call_high_12m', 
        'yield_to_call_high_ytd',
        'yield_to_call_high_6m', 
        'yield_to_call_high_3m',
        'yield_to_call_high_1m', 
        'yield_to_call_high_1w',
        'yield_to_call_low_12m', 
        'yield_to_call_low_ytd',
        'yield_to_call_low_6m', 
        'yield_to_call_low_3m', 
        'yield_to_call_low_1m',
        'yield_to_call_low_1w', 
        'yield_to_call_change_12m',
        'yield_to_call_change_ytd', 
        'yield_to_call_change_6m',
        'yield_to_call_change_3m', 
        'yield_to_call_change_1m',
        'yield_to_call_change_1w', 
        'z_spread_high_12m', 
        'z_spread_high_ytd',
        'z_spread_high_6m', 
        'z_spread_high_3m', 
        'z_spread_high_1m',
        'z_spread_high_1w', 
        'z_spread_low_12m', 
        'z_spread_low_ytd',
        'z_spread_low_6m', 
        'z_spread_low_3m', 
        'z_spread_low_1m',
        'z_spread_low_1w', 
        'z_spread_change_12m', 
        'z_spread_change_ytd',
        'z_spread_change_6m', 
        'z_spread_change_3m', 
        'z_spread_change_1m',
        'z_spread_change_1w', 
        'ytm_high_12m', 
        'ytm_high_ytd', 
        'ytm_high_6m',
        'ytm_high_3m', 
        'ytm_high_1m', 
        'ytm_high_1w', 
        'ytm_low_12m',
        'ytm_low_ytd', 
        'ytm_low_6m', 
        'ytm_low_3m', 
        'ytm_low_1m', 
        'ytm_low_1w',
        'ytm_change_12m', 
        'ytm_change_ytd', 
        'ytm_change_6m', 
        'ytm_change_3m',
        'ytm_change_1m', 
        'ytm_change_1w'
    ],
    label='y',
    dense=True,
    outlined=True
)

chart_btn = v.Btn(
    left=True,
    children=[
        v.Icon(children=['mdi-chart-line']),
        'Chart'
    ],
)

In [ ]:
param_box = v.Container(
    children=[
        v.Row(
            children=[
                v.Col(
                    cols=2,
                    children=[group_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[cap_stack_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[curr_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[chart_btn],
                    class_="my-0 py-0"
                ),
            ],
            align_content='center',
        ),
        v.Row(
            children=[
                v.Col(
                    cols=3,
                    children=[x_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=3,
                    children=[y_sel],
                    class_="my-0 py-0"
                ),
            ],
            align_content='center',
        )
    ]
)


out = w.Output()
tab = w.VBox(
    children=[
        param_box,
        out,
    ]
)

In [ ]:
@out.capture(clear_output=True)
def on_click_chart(widget, event, payload):
    currency = curr_sel.v_model
    capital_stack = cap_stack_sel.v_model
    x = x_sel.v_model
    y = y_sel.v_model
    group = group_sel.v_model

    data_ = bonds_data.query(f"currency=='{currency}' and capital_stack=='{capital_stack}'")
    fig = px.scatter(
        data_,
        x=x,
        y=y, 
        color=group,
        hover_name='name',
        height=800, 
        template='plotly_white',
        text='name'
    )

    fig.update_traces(
        textposition='bottom center',
        textfont_size=8
    )
    display(fig)
    
    
chart_btn.on_event('click', on_click_chart)